In [1]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

# Загрузка данных из файла в формате .npz
file_path = '/content/drive/MyDrive/train.npz'
data = np.load(file_path)  # Загрузка данных из файла
pics = data['data']
labels = data['labels']
# Функция для изменения размера изображений
def resize_images(pics, target_size=(64, 64)):
    resized_pics = []
    for pic in pics:
        pic_resized = cv2.resize(pic, target_size)
        resized_pics.append(pic_resized)
    return np.array(resized_pics)

# Преобразование всех изображений к заданному размеру
pics_resized = resize_images(pics, target_size=(64, 64))
# Разделение данных на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(
    pics_resized, labels, test_size=0.2, random_state=42
)

# Нормализация данных (перевод значений пикселей в диапазон от 0 до 1)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Настройка генератора данных для аугментации изображений
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
# Применение аугментации к обучающим данным
datagen.fit(x_train)

# Загрузка предобученной модели VGG16 без верхних (полносвязных) слоев
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Заморозка слоев предобученной модели, чтобы они не обучались заново
for layer in base_model.layers:
    layer.trainable = False
# Создание модели
model = Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(9, activation='softmax')
])
# Компиляция модели с использованием оптимизатора Adam
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# Обучение модели с использованием аугментированных данных
model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs=10,
    validation_data=(x_test, y_test)
)
# Оценка точности модели на тестовой выборке
accuracy = model.evaluate(x_test, y_test)
print(f"Точность модели: {accuracy[1]:.2f}")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


450/450 ━━━━━━━━━━━━━━━━━━━━ 950s 2s/step - accuracy: 0.4824 - loss: 1.5421 - val_accuracy: 0.6947 - val_loss: 0.8724
Epoch 2/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 950s 2s/step - accuracy: 0.7006 - loss: 0.8750 - val_accuracy: 0.7239 - val_loss: 0.7667
Epoch 3/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 957s 2s/step - accuracy: 0.7264 - loss: 0.7807 - val_accuracy: 0.7519 - val_loss: 0.6931
Epoch 4/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 986s 2s/step - accuracy: 0.7484 - loss: 0.7213 - val_accuracy: 0.7575 - val_loss: 0.6816
Epoch 5/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 984s 2s/step - accuracy: 0.7651 - loss: 0.6842 - val_accuracy: 0.7664 - val_loss: 0.6562
Epoch 6/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 947s 2s/step - accuracy: 0.7640 - loss: 0.6669 - val_accuracy: 0.7786 - val_loss: 0.6159
Epoch 7/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 964s 2s/step - accuracy: 0.7689 - loss: 0.6491 - val_accuracy: 0.7728 - val_loss: 0.6312
Epoch 8/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 984s 2s/step - accuracy: 0.7767 - loss: 0.6417 - val_accuracy: 0.771